In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



In [2]:
#Averías más repetitivas según ATA. El campo de título o headline es un campo que se rellena por personal del turno, no hay una estructura determinada para desarrollar por lo que no es fiable
#No voy a borrar esa columna porque quizás más adelante podamos usarla para encontrar algún tipo de patron. Por ejemplo: averías relacionadas con aire acondicionado, puede ser pack, aire, calor en cabina, etc..
#Filtraremos por ATA. Es cierto que tenemos un cierto número de campos que son nulos. 
averias = pd.read_csv("./Datos/Tablas/eventos_aviación.csv")
averias.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6464 entries, 0 to 6463
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         6464 non-null   int64 
 1   Occurred   6464 non-null   object
 2   At         6464 non-null   object
 3   Issuer     6464 non-null   int64 
 4   A/C        6464 non-null   object
 5   Station    5983 non-null   object
 6   Headline   6464 non-null   object
 7   Flight No  5296 non-null   object
 8   ATA        4524 non-null   object
 9   Advise     5495 non-null   object
dtypes: int64(2), object(8)
memory usage: 505.1+ KB


In [3]:
averias.isna().mean()*100

ID            0.000000
Occurred      0.000000
At            0.000000
Issuer        0.000000
A/C           0.000000
Station       7.441213
Headline      0.000000
Flight No    18.069307
ATA          30.012376
Advise       14.990718
dtype: float64

In [4]:
averias
#Vamos a modificar el tipo de las columnas de ocurred a datetime y at a time.

,ID,Occurred,At,Issuer,A/C,Station,Headline,Flight No,ATA,Advise
0,12185,2021-01-02,0 days 18:59:00,5121,Avión 1,PIQ,98254/1,NT421,77-00,A/C Ready
1,12194,2021-01-03,0 days 20:15:00,5121,Avión 2,PDQ,0097630/1,NT312,33-49,A/C Ready
2,12210,2021-01-04,0 days 14:13:00,5452,Avión 3,PDQ,0096673/1,NT158,52-42,A/C Ready
3,12200,2021-01-04,0 days 06:50:00,5452,Avión 3,PIQ,MAINTENANCE ACTION WITHOUT PILOT REPORT - OPER...,414,25-65,A/C Ready
4,12217,2021-01-05,0 days 07:33:00,5452,Avión 4,PIQ,OTHERS / NOT SPECIFIED - OPERATIONAL,NaN,33-00,A/C Ready
...,...,...,...,...,...,...,...,...,...,...
6459,69069,2024-07-21,0 days 20:46:00,6514,Avión 35,PIQ,LC,NT489,NaN,A/C Ready
6460,69095,2024-07-21,0 days 17:30:00,6520,Avión 41,TER,SIN ACCION DE MTTO,NT273,NaN,NaN
6461,69054,2024-07-21,0 days 14:42:00,6514,Avión 20,PDQ,BOTELLA 02 SOBRECARGO CADUCADA,NT142,NaN,A/C Ready
6462,69059,2024-07-21,0 days 18:47:00,6514,Avión 4,PDQ,CORTINA DE PASAJE FUERA DEL RIEL,NT178,NaN,A/C Ready


In [5]:
averias["Occurred"] = pd.to_datetime(averias["Occurred"], format = "%Y-%m-%d")
averias["Occurred"]

0      2021-01-02
1      2021-01-03
2      2021-01-04
3      2021-01-04
4      2021-01-05
          ...    
6459   2024-07-21
6460   2024-07-21
6461   2024-07-21
6462   2024-07-21
6463   2024-07-21
Name: Occurred, Length: 6464, dtype: datetime64[ns]

In [6]:
#Intenté convertirlo mediante expresión regular pero daba error porque no eran string, por lo que primero convierto a tipo str para después poder pasarlo a objetos time
averias["At"] = averias["At"].astype(str)
averias["At"] = averias["At"].str.extract(r"(\d{2}:\d{2}:\d{2})")[0]
averias["At"] = pd.to_datetime(averias["At"], format='%H:%M:%S').dt.time
averias["At"]

0       18:59:00
1       20:15:00
2       14:13:00
3       06:50:00
4       07:33:00
          ...   
6459    20:46:00
6460    17:30:00
6461    14:42:00
6462    18:47:00
6463    22:05:00
Name: At, Length: 6464, dtype: object

In [7]:
averias.head(10)

,ID,Occurred,At,Issuer,A/C,Station,Headline,Flight No,ATA,Advise
0,12185,2021-01-02,18:59:00,5121,Avión 1,PIQ,98254/1,NT421,77-00,A/C Ready
1,12194,2021-01-03,20:15:00,5121,Avión 2,PDQ,0097630/1,NT312,33-49,A/C Ready
2,12210,2021-01-04,14:13:00,5452,Avión 3,PDQ,0096673/1,NT158,52-42,A/C Ready
3,12200,2021-01-04,06:50:00,5452,Avión 3,PIQ,MAINTENANCE ACTION WITHOUT PILOT REPORT - OPER...,414,25-65,A/C Ready
4,12217,2021-01-05,07:33:00,5452,Avión 4,PIQ,OTHERS / NOT SPECIFIED - OPERATIONAL,NaN,33-00,A/C Ready
5,12229,2021-01-06,07:32:00,5452,Avión 5,PDQ,46331/1,NT106,21-00,A/C Ready
6,12274,2021-01-08,14:57:00,3444,Avión 6,PDQ,02377/1,NaN,24-00,A/C Ready
7,12276,2021-01-08,17:47:00,5479,Avión 1,PDQ,98263/D1,NT311,27-50,A/C Ready
8,12268,2021-01-08,08:27:00,3444,Avión 7,PIQ,T/S Y TRABAJOS PROGRAMADOS,NaN,34-57,A/C Ready
9,12266,2021-01-08,08:04:00,3444,Avión 5,PDQ,CREW AVISA LAS 07:50,106,21-00,A/C Ready


In [8]:
#Vamos a borrar varias columnas en las que los datos no nos aportan nada
averias.drop(columns= ["Issuer","Advise","Headline"], inplace = True)
averias

,ID,Occurred,At,A/C,Station,Flight No,ATA
0,12185,2021-01-02,18:59:00,Avión 1,PIQ,NT421,77-00
1,12194,2021-01-03,20:15:00,Avión 2,PDQ,NT312,33-49
2,12210,2021-01-04,14:13:00,Avión 3,PDQ,NT158,52-42
3,12200,2021-01-04,06:50:00,Avión 3,PIQ,414,25-65
4,12217,2021-01-05,07:33:00,Avión 4,PIQ,NaN,33-00
...,...,...,...,...,...,...,...
6459,69069,2024-07-21,20:46:00,Avión 35,PIQ,NT489,NaN
6460,69095,2024-07-21,17:30:00,Avión 41,TER,NT273,NaN
6461,69054,2024-07-21,14:42:00,Avión 20,PDQ,NT142,NaN
6462,69059,2024-07-21,18:47:00,Avión 4,PDQ,NT178,NaN


In [9]:
#Tenemos un 30% de los datos de las ATA con NaN
#Como nos interesan las averías que causan mayor retraso a la hora de coger un vuelo vamos hacer una query que identifique que eventos son y sacar una tabla combinada de ambas. 
#Pero antes tenemos que limpiar y tratar los datos de la tabla de minutos de retraso
retrasos = pd.read_csv("./Datos/Tablas/minutosretrasoporavion.csv")
retrasos

,Fecha,EVENTO,A/C,ENCABEZADO EVENTO,Minutos
0,2022-07-22 00:00:00,32171.0,Avión 6,NOSE LANDING GEAR,1541
1,2022-08-16 00:00:00,33504.0,Avión 12,FLT CTRL NO DISPATCH ADS 3 HTR FAIL,1492
2,2022-09-07 00:00:00,34578.0,Avión 6,PTV IMPACTO DE PÁJARO EN ZAZ,1481
3,2023-07-23 00:00:00,50049.0,Avión 12,FUGA DE HYD EN FRENO N2,1263
4,2022-01-04 00:00:00,23928.0,Avión 15,ENGINE1 ITT RISES TO 946C,1167
...,...,...,...,...,...
32459,2023-02-13 00:00:00,41962.0,Avión 11,51199501,0
32460,2023-02-11 00:00:00,41886.0,Avión 28,51202341,0
32461,2021-03-03 00:00:00,13185.0,Avión 1,97773123,0
32462,2021-04-24 00:00:00,14082.0,Avión 10,501003741,0


In [10]:
retrasos["Fecha"] = pd.to_datetime(retrasos["Fecha"], errors="coerce") # Coerce pasa los valores que no coincidan a nat
retrasos["Fecha"] = retrasos["Fecha"].dt.date #cogemos solo la parte de la fecha
retrasos["Fecha"] = pd.to_datetime(retrasos["Fecha"]) # convertimos la columna a datetime sin la hora 
retrasos


,Fecha,EVENTO,A/C,ENCABEZADO EVENTO,Minutos
0,2022-07-22,32171.0,Avión 6,NOSE LANDING GEAR,1541
1,2022-08-16,33504.0,Avión 12,FLT CTRL NO DISPATCH ADS 3 HTR FAIL,1492
2,2022-09-07,34578.0,Avión 6,PTV IMPACTO DE PÁJARO EN ZAZ,1481
3,2023-07-23,50049.0,Avión 12,FUGA DE HYD EN FRENO N2,1263
4,2022-01-04,23928.0,Avión 15,ENGINE1 ITT RISES TO 946C,1167
...,...,...,...,...,...
32459,2023-02-13,41962.0,Avión 11,51199501,0
32460,2023-02-11,41886.0,Avión 28,51202341,0
32461,2021-03-03,13185.0,Avión 1,97773123,0
32462,2021-04-24,14082.0,Avión 10,501003741,0


In [11]:
#Ahora gestionaremos la columna Eventos para que coincida con los campos de ID de la tabla eventos, lo pasamos a int
retrasos["EVENTO"] = retrasos["EVENTO"].astype(int)
retrasos["EVENTO"]

0        32171
1        33504
2        34578
3        50049
4        23928
         ...  
32459    41962
32460    41886
32461    13185
32462    14082
32463    29781
Name: EVENTO, Length: 32464, dtype: int32

In [12]:
retrasos

,Fecha,EVENTO,A/C,ENCABEZADO EVENTO,Minutos
0,2022-07-22,32171,Avión 6,NOSE LANDING GEAR,1541
1,2022-08-16,33504,Avión 12,FLT CTRL NO DISPATCH ADS 3 HTR FAIL,1492
2,2022-09-07,34578,Avión 6,PTV IMPACTO DE PÁJARO EN ZAZ,1481
3,2023-07-23,50049,Avión 12,FUGA DE HYD EN FRENO N2,1263
4,2022-01-04,23928,Avión 15,ENGINE1 ITT RISES TO 946C,1167
...,...,...,...,...,...
32459,2023-02-13,41962,Avión 11,51199501,0
32460,2023-02-11,41886,Avión 28,51202341,0
32461,2021-03-03,13185,Avión 1,97773123,0
32462,2021-04-24,14082,Avión 10,501003741,0


In [13]:
#Guardamos las tablas con los nuevos cambios
#Guardar nuevo dataframe
averias.to_csv("./Datos/Tablas/averias.csv", index=False)

In [14]:
#Guardar nuevo dataframe
retrasos.to_csv("./Datos/Tablas/retrasos.csv", index=False)

In [15]:
#Tenemos un 30% de los datos de las ATA con NaN
#Como nos interesan las averías que causan mayor retraso a la hora de coger un vuelo vamos hacer una query que identifique que eventos son y sacar una tabla combinada de ambas. 

In [16]:
import sqlite3
import csv

In [17]:
#creamos base datos
conector = sqlite3.connect("base_de_datos.db")
query = conector.cursor()

# 2. Crear las tablas
query.execute('''
    CREATE TABLE IF NOT EXISTS averias (
        ID INTEGER PRIMARY KEY,
        Occurred TEXT,
        At TEXT,
        AC TEXT,
        Station TEXT,
        FlightNo TEXT,
        ATA TEXT)''')

query.execute('''
    CREATE TABLE IF NOT EXISTS retrasos (
        Fecha TEXT,
        EVENTO INTEGER,
        AC TEXT,
        EncabezadoEvento TEXT,
        Minutos INTEGER,
        PRIMARY KEY (EVENTO),
        FOREIGN KEY (EVENTO) REFERENCES eventos(ID))''')



# Insertar datos en la tabla averias desde un archivo CSV
with open("./Datos/Tablas/averias.csv", "r", encoding = "latin1") as f:
    reader = csv.reader(f)
    next(reader)  # Omitir la cabecera si existe
    for row in reader:
        query.execute('''
            INSERT OR REPLACE INTO averias (ID, Occurred, At, AC, Station, FlightNo, ATA)
            VALUES (?, ?, ?, ?, ?, ?, ?)
        ''', row)

# Insertar datos en la tabla retrasos desde un archivo CSV
with open("./Datos/Tablas/retrasos.csv", "r", encoding ="latin1") as f:
    reader = csv.reader(f)
    next(reader)  # Omitir la cabecera si existe
    for row in reader:
        query.execute('''
            INSERT OR REPLACE INTO retrasos (Fecha, EVENTO, AC, EncabezadoEvento, Minutos)
            VALUES (?, ?, ?, ?, ?)
        ''', row)


# Guardar los cambios
conector.commit()

query.execute('''
              CREATE TABLE IF NOT EXISTS eventos_con_retrasos AS
              SELECT averias.ID, averias.Occurred, averias.At, averias.AC, averias.Station, averias.FlightNo, averias.ATA, retrasos.Minutos
              FROM averias
              LEFT JOIN retrasos ON averias.ID = retrasos.EVENTO''')

#Consultar si se han cargado bien los datos en la nueva tabla eventos_con_retrasos

query.execute('''SELECT * FROM eventos_con_retrasos''')
filas = query.fetchall()
for fila in filas:
    print(fila)


# Guardar como archivo CSV
with open("./Datos/Tablas/eventos_con_retrasos.csv", "w", newline="", encoding = "latin1") as f:
    writer = csv.writer(f)
    writer.writerow([description[0] for description in query.description]) # Escribir la cabecera del CSV
    writer.writerows(filas)# Escribir las filas

# Guardar los cambios
conector.commit()

# Cerrar la conexión
query.close()
conector.close()


(12185, '2021-01-02', '18:59:00', 'AviÃ³n 1', 'PIQ', '98254/1', 'NT421', '77-00', 38)
(12194, '2021-01-03', '20:15:00', 'AviÃ³n 2', 'PDQ', '0097630/1', 'NT312', '33-49', 15)
(12200, '2021-01-04', '06:50:00', 'AviÃ³n 3', 'PIQ', 'MAINTENANCE ACTION WITHOUT PILOT REPORT - OPERATIONAL', '414', '25-65', 3)
(12210, '2021-01-04', '14:13:00', 'AviÃ³n 3', 'PDQ', '0096673/1', 'NT158', '52-42', 12)
(12217, '2021-01-05', '07:33:00', 'AviÃ³n 4', 'PIQ', 'OTHERS / NOT SPECIFIED - OPERATIONAL', '', '33-00', 8)
(12229, '2021-01-06', '07:32:00', 'AviÃ³n 5', 'PDQ', '46331/1', 'NT106', '21-00', 39)
(12266, '2021-01-08', '08:04:00', 'AviÃ³n 5', 'PDQ', 'CREW AVISA LAS 07:50', '106', '21-00', 16)
(12268, '2021-01-08', '08:27:00', 'AviÃ³n 7', 'PIQ', 'T/S Y TRABAJOS PROGRAMADOS', '', '34-57', 30)
(12274, '2021-01-08', '14:57:00', 'AviÃ³n 6', 'PDQ', '02377/1', '', '24-00', 64)
(12276, '2021-01-08', '17:47:00', 'AviÃ³n 1', 'PDQ', '98263/D1', 'NT311', '27-50', 50)
(12290, '2021-01-09', '16:52:00', 'AviÃ³n 8', 'PD